## Import modules, set options

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import math
import gc
import copy

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor
import os


from Utils import *
# os.listdir('../input/imputed')

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)

In [3]:
submission_csv = pd.read_csv(f'{Config.DATA_PATH}\\sample_submission.csv', index_col='id')

C:\ProgramFiles\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Submission Model


In [4]:
test_csv = pd.read_csv(f'{Config.DATA_PATH}\\test.csv', index_col='id', dtype=train_dtypes)
test_csv['molecule_index'] = test_csv['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')
test_csv = test_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type']]
test_csv.head(10)

,molecule_index,atom_index_0,atom_index_1,type
id,,,,
4658147,4,2,0,2JHC
4658148,4,2,1,1JHC
4658149,4,2,3,3JHH
4658150,4,3,0,1JHC
4658151,4,3,1,2JHC
4658152,15,3,0,1JHC
4658153,15,3,2,3JHC
4658154,15,3,4,2JHH
4658155,15,3,5,2JHH


In [5]:
from sklearn.model_selection import GroupKFold, train_test_split
def train_and_predict_for_one_coupling_type(coupling_type, submission, n_folds=5, n_splits=5, random_state=128):
    model_type =  'lgb' #lgb  cat
    print(f'*** Training Model for {coupling_type} ***')
    XY_Data = pd.read_csv(f'{Config.INPUT_XY}/{coupling_type}.csv', index_col=0)
    X_data, y_data = build_x_y_data(XY_Data)    
    
    columns = X_data.columns 
    XY_Test = pd.read_csv(f'{Config.INPUT_XY}/test_{coupling_type}.csv', index_col=0)
    X_test, _ = build_x_y_data(XY_Test)
    y_pred = np.zeros(X_test.shape[0], dtype='float32')

    train_csv = load_train()

    cv_score = 0
    
    if n_folds > n_splits:
        n_splits = n_folds
    
    #kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kfold = GroupKFold(n_splits=n_splits)
    
    
    cur_train_csv = train_csv[train_csv.type == coupling_type]
    groups  = cur_train_csv.molecule_index.values

    for fold, (train_index, val_index) in enumerate(kfold.split(X_data, y_data, groups=groups)):
        if fold >= n_folds:
            break

        #X_train, X_val = X_data[train_index], X_data[val_index]
        #y_train, y_val = y_data[train_index], y_data[val_index]
        X_train, X_val = X_data[columns].iloc[train_index], X_data[columns].iloc[val_index]
        y_train, y_val = y_data.iloc[train_index], y_data.iloc[val_index]

        
        model = ""
        categorical_features = [col for col in X_train if col.startswith('atom_')]
        if model_type == 'lgb':

            model = LGBMRegressor(**Config.LGB_PARAMS, n_estimators=15000, n_jobs = 5)
            #model = LGBMRegressor(**LGB_PARAMS, n_estimators=6000, n_jobs = -1)
            model.fit(X_train, y_train, 
                eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
                verbose=100, early_stopping_rounds=1000,
                     categorical_feature = categorical_features)
        
        if model_type == 'cat': 
            model = CatBoostRegressor(eval_metric='MAE', **CAT_PARAMS, loss_function='MAE')
            X_trainGlob = X_train
            model.fit(X_train, y_train, eval_set=((X_val, y_val)), 
                      cat_features = categorical_features,
                      use_best_model=True,
                      verbose=False)

        y_val_pred = model.predict(X_val)
        val_score = np.log(mean_absolute_error(y_val, y_val_pred))
        print(f'{coupling_type} Fold {fold}, logMAE: {val_score}')
        
        cv_score += val_score / n_folds
        y_pred += model.predict(X_test) / n_folds
        
    submission.loc[test_csv['type'] == coupling_type, 'scalar_coupling_constant'] = y_pred
    return cv_score

Let's build a separate model for each type of coupling. Dataset is split into 5 pieces and in this kernel we will use only 3 folds for speed up.

Main tuning parameter is the number of atoms. I took good numbers, but accuracy can be improved a bit by tuning them for each type.

Checking cross-validation scores for each type:

In [6]:
N_FOLDS = 5
submission = submission_csv.copy()

cv_scores = {}
for coupling_type in Config.MODEL_PARAMS.keys():
    cv_score = train_and_predict_for_one_coupling_type(
        coupling_type, submission, n_folds=N_FOLDS)
    cv_scores[coupling_type] = cv_score

*** Training Model for 1JHN ***


C:\ProgramFiles\Anaconda\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['atom_10', 'atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7', 'atom_8', 'atom_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 1000 rounds.
[100]	training's l1: 0.537618	valid_1's l1: 0.585336
[200]	training's l1: 0.413442	valid_1's l1: 0.490275
[300]	training's l1: 0.351078	valid_1's l1: 0.451444
[400]	training's l1: 0.307937	valid_1's l1: 0.426764
[500]	training's l1: 0.275382	valid_1's l1: 0.409766
[600]	training's l1: 0.249522	valid_1's l1: 0.397709
[700]	training's l1: 0.227688	valid_1's l1: 0.387508
[800]	training's l1: 0.21004	valid_1's l1: 0.380222
[900]	training's l1: 0.194102	valid_1's l1: 0.374254
[1000]	training's l1: 0.180395	valid_1's l1: 0.369353
[1100]	training's l1: 0.168497	valid_1's l1: 0.365107
[1200]	training's l1: 0.157691	valid_1's l1: 0.361835
[1300]	training's l1: 0.148041	valid_1's l1: 0.359033
[1400]	training's l1: 0.139338	valid_1's l1: 0.356513
[1500]	training's l1: 0.131388	valid_1's l1: 0.354285
[1600]	training's l1: 0.123978	valid_1's l1: 0.352345
[1700]	training's l1: 0.117423	valid_1's l1: 0.350892
[1800]	training's l1: 0.1112

[14900]	training's l1: 0.00201781	valid_1's l1: 0.331873
[15000]	training's l1: 0.00197249	valid_1's l1: 0.331868
Did not meet early stopping. Best iteration is:
[15000]	training's l1: 0.00197249	valid_1's l1: 0.331868
1JHN Fold 0, logMAE: -1.1030186004080824


C:\ProgramFiles\Anaconda\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['atom_10', 'atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7', 'atom_8', 'atom_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


...

Training until validation scores don't improve for 1000 rounds.
[100]	training's l1: 0.247143	valid_1's l1: 0.252515
[200]	training's l1: 0.203008	valid_1's l1: 0.213152
[300]	training's l1: 0.179158	valid_1's l1: 0.193106
[400]	training's l1: 0.163232	valid_1's l1: 0.180301
[500]	training's l1: 0.150657	valid_1's l1: 0.17038
[600]	training's l1: 0.140869	valid_1's l1: 0.163114
[700]	training's l1: 0.132597	valid_1's l1: 0.156996
[800]	training's l1: 0.125393	valid_1's l1: 0.151856
[900]	training's l1: 0.119193	valid_1's l1: 0.147607
[1000]	training's l1: 0.113813	valid_1's l1: 0.14402
[1100]	training's l1: 0.10911	valid_1's l1: 0.141019
[1200]	training's l1: 0.104801	valid_1's l1: 0.138334
[1300]	training's l1: 0.100936	valid_1's l1: 0.135918
[1400]	training's l1: 0.0974138	valid_1's l1: 0.133801
[1500]	training's l1: 0.0941216	valid_1's l1: 0.131868
[1600]	training's l1: 0.0910172	valid_1's l1: 0.130242
[1700]	training's l1: 0.0880965	valid_1's l1: 0.128612
[1800]	training's l1: 0.08

[14900]	training's l1: 0.0125055	valid_1's l1: 0.101944
[15000]	training's l1: 0.0123849	valid_1's l1: 0.101921
Did not meet early stopping. Best iteration is:
[15000]	training's l1: 0.0123849	valid_1's l1: 0.101921
3JHN Fold 3, logMAE: -2.28355338493762


C:\ProgramFiles\Anaconda\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['atom_10', 'atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7', 'atom_8', 'atom_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 1000 rounds.
[100]	training's l1: 0.246235	valid_1's l1: 0.257501
[200]	training's l1: 0.201345	valid_1's l1: 0.215582
[300]	training's l1: 0.177415	valid_1's l1: 0.194626
[400]	training's l1: 0.160859	valid_1's l1: 0.180704
[500]	training's l1: 0.148345	valid_1's l1: 0.170813
[600]	training's l1: 0.138207	valid_1's l1: 0.162923
[700]	training's l1: 0.130189	valid_1's l1: 0.157048
[800]	training's l1: 0.12356	valid_1's l1: 0.15228
[900]	training's l1: 0.117629	valid_1's l1: 0.148091
[1000]	training's l1: 0.112371	valid_1's l1: 0.144632
[1100]	training's l1: 0.107777	valid_1's l1: 0.141745
[1200]	training's l1: 0.103663	valid_1's l1: 0.139335
[1300]	training's l1: 0.100042	valid_1's l1: 0.137185
[1400]	training's l1: 0.0964206	valid_1's l1: 0.135067
[1500]	training's l1: 0.0931597	valid_1's l1: 0.133077
[1600]	training's l1: 0.0901067	valid_1's l1: 0.131298
[1700]	training's l1: 0.0873327	valid_1's l1: 0.129776
[1800]	training's l1: 0.0

[15000]	training's l1: 0.0123662	valid_1's l1: 0.103232
Did not meet early stopping. Best iteration is:
[15000]	training's l1: 0.0123662	valid_1's l1: 0.103232
3JHN Fold 4, logMAE: -2.270774089701595


In [7]:
pd.DataFrame({'type': list(cv_scores.keys()), 'cv_score': list(cv_scores.values())})

,type,cv_score
0,1JHN,-1.106502
1,1JHC,-0.390804
2,2JHH,-1.966835
3,2JHN,-2.076056
4,2JHC,-1.387011
5,3JHH,-2.006237
6,3JHC,-1.260956
7,3JHN,-2.272088


And cv mean score:

In [8]:
np.mean(list(cv_scores.values()))

-1.558311090702058

Sanity check for all cells to be filled with predictions:

In [9]:
submission[submission['scalar_coupling_constant'] == 0].shape

(0, 1)

In [10]:
submission.head(10)

,scalar_coupling_constant
id,
4658147,12.797239
4658148,164.063110
4658149,2.448566
4658150,164.063110
4658151,12.797239
4658152,91.457123
4658153,2.108005
4658154,-7.776263
4658155,-9.894598


In [11]:
submission.to_csv(f'{Config.SUBMISSIONS_PATH}/submission.csv')

## a!!!